<a href="https://colab.research.google.com/github/eddson90/tareas_data_science/blob/main/statistical-learning-2/proyecto-final/parte3/Parte3_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Parte 3: Red Neuronal Recurrente (RNN)**

## Proyecto Final: Implementación de Modelos de Deep Learning
---

Statistical Learning II

Universidad Galileo

Eddson Sierra

## Carga de librerías

In [2]:
pip install pickle5

     |████████████████████████████████| 132 kB 5.2 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219303 sha256=a3b0db7f751510f9d854169ce3a662e704241fdd8762842572aa8f0d6e862468
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [3]:
import pickle5 as pickle

In [4]:
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
%matplotlib inline

import string
import os
import glob
from PIL import Image
from time import time

from keras import Input, layers
from keras import optimizers
from tensorflow.keras.optimizers import Adam
 
from keras.preprocessing import sequence
from keras.preprocessing import image
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Embedding, Dense, Activation, Flatten, Reshape, Dropout
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from tensorflow.keras.utils import to_categorical

## Carga de datos y preprocesamiento

In [5]:
images_path = '/content/drive/MyDrive/Colab Notebooks/Proyecto Final SL2/Parte2_CNN/img_data_file.pkl'
tweet_content_path = '/content/drive/MyDrive/Colab Notebooks/Proyecto Final SL2/Parte2_CNN/tweet_content_file.pkl'

In [6]:
def load_from_pk(file_name):
  pickle_file = open(file_name, "rb")
  #objects = []
  while True:
      try:
          #objects.append(pickle.load(pickle_file))
          salida = pickle.load(pickle_file)
      except EOFError:
          break
  pickle_file.close()

  return salida

In [7]:
images = np.array(load_from_pk(images_path))
captions = np.array(load_from_pk(tweet_content_path)).reshape(-1,1)

In [8]:
images.shape, captions.shape

((5574, 128, 128, 3), (5574, 1))

In [9]:
captions[0:4]

array([['¡Hola, @TopoGigo6! Gracias por compartir. Pertenece a la familia #Corinnidae probablemente. No son consideradas de importancia médica #NIM✅ Saludos https://t.co/hwXi74WiLs'],
       ['¡Hola, @Elwestrand! Gracias por compartir. Pertenece a la familia #Thomisidae, posiblemente del género «Thomisius». No son consideradas de importancia médica #NIM✅ Saludos https://t.co/vUaVvkMHOn'],
       ['¡Hola, @ddiazgar76! Gracias por la consulta. Pertenece a la familia #Oxyopidae, del género #Peucetia probablemente. Se les conoce como arañas lince verdes. No son consideradas de importancia médica #NIM✅ Le hacen falta un par de patas. Saludos https://t.co/UMjC3fIxTF'],
       ['¡Hola, @adukation! Gracias por tu consulta. Pertenece a la familia #Vaejovidae, posiblemente del género #Vaejovis. No son considerados de importancia médica #NIM✅ Sí, es posible que haya más, por el hecho de ser la CDMX zona de distribución natural de 2 especies de este género https://t.co/InMQqvn8xC']],
      dtype='

#### Pasar lista a string

Creamos una función para pasar la lista a string y la aplicamos para cada elemento de la lista de captions (que es una lista en sí misma):

In [10]:
def listToString(s): 
    str1 = " " 
    return (str1.join(s))

In [11]:
captions_list_str = []
for i in range(len(captions)):
  caption_str = listToString(captions[i])
  captions_list_str.append(caption_str)

In [12]:
captions_list_str[0:4]

['¡Hola, @TopoGigo6! Gracias por compartir. Pertenece a la familia #Corinnidae probablemente. No son consideradas de importancia médica #NIM✅ Saludos https://t.co/hwXi74WiLs',
 '¡Hola, @Elwestrand! Gracias por compartir. Pertenece a la familia #Thomisidae, posiblemente del género «Thomisius». No son consideradas de importancia médica #NIM✅ Saludos https://t.co/vUaVvkMHOn',
 '¡Hola, @ddiazgar76! Gracias por la consulta. Pertenece a la familia #Oxyopidae, del género #Peucetia probablemente. Se les conoce como arañas lince verdes. No son consideradas de importancia médica #NIM✅ Le hacen falta un par de patas. Saludos https://t.co/UMjC3fIxTF',
 '¡Hola, @adukation! Gracias por tu consulta. Pertenece a la familia #Vaejovidae, posiblemente del género #Vaejovis. No son considerados de importancia médica #NIM✅ Sí, es posible que haya más, por el hecho de ser la CDMX zona de distribución natural de 2 especies de este género https://t.co/InMQqvn8xC']

#### Convertir a minúsculas

Convertimos la lista de captions a minúsculas:

In [179]:
captions_list_clean = []
for i in range(len(captions_list_str)):
        desc = captions_list_str[i]
        desc = desc.split()
        desc = [word.lower() for word in desc]
        captions_list_clean.append(' '.join(desc))

In [181]:
captions_list_clean[0:4]

['¡hola, @topogigo6! gracias por compartir. pertenece a la familia #corinnidae probablemente. no son consideradas de importancia médica #nim✅ saludos https://t.co/hwxi74wils',
 '¡hola, @elwestrand! gracias por compartir. pertenece a la familia #thomisidae, posiblemente del género «thomisius». no son consideradas de importancia médica #nim✅ saludos https://t.co/vuavvkmhon',
 '¡hola, @ddiazgar76! gracias por la consulta. pertenece a la familia #oxyopidae, del género #peucetia probablemente. se les conoce como arañas lince verdes. no son consideradas de importancia médica #nim✅ le hacen falta un par de patas. saludos https://t.co/umjc3fixtf',
 '¡hola, @adukation! gracias por tu consulta. pertenece a la familia #vaejovidae, posiblemente del género #vaejovis. no son considerados de importancia médica #nim✅ sí, es posible que haya más, por el hecho de ser la cdmx zona de distribución natural de 2 especies de este género https://t.co/inmqqvn8xc']

#### Remover usuario y url con regex

Removemos los nombres de usuario y las urls presentes en las captions a través de expresiones regulares:

In [182]:
import re

In [183]:
patron_user = r"@([a-z]*)?([0-9]*)?(!|_|-)?"
patron_url = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"

In [184]:
for i in range(len(captions_list_clean)):
  try:
    match_user = re.search(patron_user, captions_list_clean[i]).group()
    match_url = re.search(patron_url, captions_list_clean[i]).group()
    captions_list_clean[i] = captions_list_clean[i].replace(match_user,'')
    captions_list_clean[i] = captions_list_clean[i].replace(match_url,'')
  except:
    try:
      match_user = re.search(patron_user, captions_list_clean[i]).group()
      captions_list_clean[i] = captions_list_clean[i].replace(match_user,'')
    except:
      try:
        match_url = re.search(patron_url, captions_list_clean[i]).group()
        captions_list_clean[i] = captions_list_clean[i].replace(match_url,'')
      except:
        continue 

In [185]:
captions_list_clean[0:4]

['¡hola,  gracias por compartir. pertenece a la familia #corinnidae probablemente. no son consideradas de importancia médica #nim✅ saludos ',
 '¡hola,  gracias por compartir. pertenece a la familia #thomisidae, posiblemente del género «thomisius». no son consideradas de importancia médica #nim✅ saludos ',
 '¡hola,  gracias por la consulta. pertenece a la familia #oxyopidae, del género #peucetia probablemente. se les conoce como arañas lince verdes. no son consideradas de importancia médica #nim✅ le hacen falta un par de patas. saludos ',
 '¡hola,  gracias por tu consulta. pertenece a la familia #vaejovidae, posiblemente del género #vaejovis. no son considerados de importancia médica #nim✅ sí, es posible que haya más, por el hecho de ser la cdmx zona de distribución natural de 2 especies de este género ']

### Remoción de emojis

Utilizando la librería demoji, creamos una función que permite extraer los emojis utilizados en un texto y la utilizamos para remover todos los emojis en las captions:

In [85]:
pip install demoji

     |████████████████████████████████| 42 kB 723 kB/s 


In [151]:
import demoji
demoji.download_codes()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  This is separate from the ipykernel package so we can avoid doing imports until


In [186]:
for i in range(len(captions_list_clean)):
  try:
    emoji_dict = demoji.findall(captions_list_clean[i])
    for key in emoji_dict:
      captions_list_clean[i] = captions_list_clean[i].replace(key,'')
  except:
    continue

In [187]:
captions_list_clean[0:4]

['¡hola,  gracias por compartir. pertenece a la familia #corinnidae probablemente. no son consideradas de importancia médica #nim saludos ',
 '¡hola,  gracias por compartir. pertenece a la familia #thomisidae, posiblemente del género «thomisius». no son consideradas de importancia médica #nim saludos ',
 '¡hola,  gracias por la consulta. pertenece a la familia #oxyopidae, del género #peucetia probablemente. se les conoce como arañas lince verdes. no son consideradas de importancia médica #nim le hacen falta un par de patas. saludos ',
 '¡hola,  gracias por tu consulta. pertenece a la familia #vaejovidae, posiblemente del género #vaejovis. no son considerados de importancia médica #nim sí, es posible que haya más, por el hecho de ser la cdmx zona de distribución natural de 2 especies de este género ']

### Remover otros signos de puntuación

In [189]:
punc = '''!¡()-[]{};:'"\,<>./?@#$%^&*_~'''
for i in range(len(captions_list_str)):
  for ele in captions_list_clean[i]:
      if ele in punc:
          captions_list_clean[i] = captions_list_clean[i].replace(ele, "")

In [190]:
captions_list_clean[0:4]

['hola  gracias por compartir pertenece a la familia corinnidae probablemente no son consideradas de importancia médica nim saludos ',
 'hola  gracias por compartir pertenece a la familia thomisidae posiblemente del género «thomisius» no son consideradas de importancia médica nim saludos ',
 'hola  gracias por la consulta pertenece a la familia oxyopidae del género peucetia probablemente se les conoce como arañas lince verdes no son consideradas de importancia médica nim le hacen falta un par de patas saludos ',
 'hola  gracias por tu consulta pertenece a la familia vaejovidae posiblemente del género vaejovis no son considerados de importancia médica nim sí es posible que haya más por el hecho de ser la cdmx zona de distribución natural de 2 especies de este género ']

### Creación de un vocabulario